## Processing all the row data files -> select best sensors and process these sensors data in the new file for ML

In [1]:
import numpy as np
import pandas as pd
import os
import time

import matplotlib.pyplot as plt

from scipy.signal import savgol_filter

import csv
from utils import TP_detect,baseline_als

file_list = ["./RawData_Baby_Sensorpatch2/2021-09-08_15-21-54_2021-09-08_15-24-06_sensor_input_5000_1631107314600325.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_15-21-54_2021-09-08_15-26-17_sensor_input_5000_1631107314600325.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_15-21-54_2021-09-08_15-28-29_sensor_input_5000_1631107314600325.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_15-21-54_2021-09-08_15-30-40_sensor_input_5000_1631107314600325.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_15-21-54_2021-09-08_15-32-12_sensor_input_3516_1631107314600325.csv",
         
             "./RawData_Baby_Sensorpatch2/2021-09-08_15-32-19_2021-09-08_15-34-31_sensor_input_5000_1631107939624088.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_15-32-19_2021-09-08_15-36-42_sensor_input_5000_1631107939624088.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_15-32-19_2021-09-08_15-38-54_sensor_input_5000_1631107939624088.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_15-32-19_2021-09-08_15-41-05_sensor_input_5000_1631107939624088.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_15-32-19_2021-09-08_15-42-23_sensor_input_2979_1631107939624088.csv",
         
             "./RawData_Baby_Sensorpatch2/2021-09-08_15-42-30_2021-09-08_15-44-42_sensor_input_5000_1631108550428185.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_15-42-30_2021-09-08_15-46-53_sensor_input_5000_1631108550428185.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_15-42-30_2021-09-08_15-49-05_sensor_input_5000_1631108550428185.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_15-42-30_2021-09-08_15-51-16_sensor_input_5000_1631108550428185.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_15-42-30_2021-09-08_15-52-23_sensor_input_2560_1631108550428185.csv",
         
             "./RawData_Baby_Sensorpatch2/2021-09-08_15-52-28_2021-09-08_15-54-40_sensor_input_5000_1631109148920218.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_15-52-28_2021-09-08_15-56-52_sensor_input_5000_1631109148920218.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_15-52-28_2021-09-08_15-59-03_sensor_input_5000_1631109148920218.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_15-52-28_2021-09-08_16-01-14_sensor_input_5000_1631109148920218.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_15-52-28_2021-09-08_16-02-05_sensor_input_1937_1631109148920218.csv",
         
             "./RawData_Baby_Sensorpatch2/2021-09-08_16-02-09_2021-09-08_16-04-21_sensor_input_5000_1631109729643834.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_16-02-09_2021-09-08_16-06-32_sensor_input_5000_1631109729643834.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_16-02-09_2021-09-08_16-08-44_sensor_input_5000_1631109729643834.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_16-02-09_2021-09-08_16-10-55_sensor_input_5000_1631109729643834.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_16-02-09_2021-09-08_16-13-07_sensor_input_5000_1631109729643834.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_16-02-09_2021-09-08_16-14-15_sensor_input_2585_1631109729643834.csv"]

# baseline
lam = 10e3
p = 0.5
# savFilter
win_len = 9
sav_order = 5
# adapt window length is 300
adapt_Window = 300
i_file_index = 0
#filepath = 'F:/PyCases/Jupter/Ventilator/LSTM_training/Train_data/traindata.csv'
#os.remove(filepath)
filepath = './Train_data'
testset = []
for i_file in file_list:
    i_file_index += 1
    input_data = pd.read_csv(i_file).to_numpy()

    sensors_value = np.clip(input_data[:,1:],a_min = 0, a_max = 4095)    # 36sensors value
    timelineinsec = input_data[:,0]/1000000    # time in second, ndarray

    num_messdata, num_sensors = sensors_value.shape

    time_diff = np.diff(timelineinsec).mean()   # get the mean value of the deita time(in two sensor signal)
    sample_rate = 1/time_diff        # sample rate, Hz ~38Hz


    FC_peak_all, FC_tough_all, CC_all = [],[],[]
    for i_sensor in range(num_sensors):
        Sav_filter_win =savgol_filter(sensors_value[:,i_sensor],win_len,sav_order,mode='nearest')
        baseline_sensor = baseline_als(Sav_filter_win,lam,p)
        norm_sensor = Sav_filter_win - baseline_sensor
        FC_peak_index,FC_tough_index,FC_peak,FC_tough,CC = TP_detect(norm_sensor)
    
        FC_peak_all.append(FC_peak)
        FC_tough_all.append(FC_tough)
        CC_all.append(CC)

    CC_all = np.array(CC_all,dtype = int).reshape(-1)

    # according to the respiration frequency 1-2Hz,
    upbreath_freq = 1.5     # the max breath freq by infants
    lowbreath_freq = 0.8    # incase Artifact or Apenea, the min breath freq by infants

    # boundary for CC, Hz
    lowbound_CC = (2 * lowbreath_freq * 2) * num_messdata / sample_rate  # 2 means peak and tough
    upbound_CC =  (2 * upbreath_freq * 2) * num_messdata / sample_rate   # can max 2 times of the max breath frequency allowable
    up_CC = np.arange(num_sensors)[CC_all<upbound_CC]
    low_CC = np.arange(num_sensors)[CC_all>lowbound_CC]

    CC_match = np.array([x for x in low_CC if x in up_CC])     # selected sensor in CC boundary

    if len(CC_match) < 1:
        continue
    else:
        TP_index_all = []
        last_start_all = []
        Inverse_sensor = []
        CC_match_ = CC_match.copy()
        
        for i_match in CC_match:
            
            start_adapt = 0
            TP_diff = 50
            while TP_diff > 30:
                samp_Sav_filter_win =savgol_filter(sensors_value[:,i_match][start_adapt:start_adapt+adapt_Window],win_len,sav_order,mode='nearest')
                samp_baseline_sensor = baseline_als(samp_Sav_filter_win,lam,p)
                samp_nonoff_sensor = samp_Sav_filter_win - samp_baseline_sensor
                
                low_samp = np.quantile(samp_nonoff_sensor,0.25)
                up_samp = np.quantile(samp_nonoff_sensor,0.75)
                IQR = up_samp - low_samp
                
                samp_norm = (samp_nonoff_sensor + 2*IQR) / 4 / IQR
                samp_norm = np.clip(samp_norm,0,1)
                
                TP_index_pot = []
                for i_point in range(1,(adapt_Window-5)):
                    slope_1 = samp_nonoff_sensor[i_point] - samp_nonoff_sensor[i_point-1]
                    slope_2 = samp_nonoff_sensor[i_point+1] - samp_nonoff_sensor[i_point]
                # if change point
                    if slope_1*slope_2 > 0:
                        continue
                    else:
                        slope_3 = samp_nonoff_sensor[i_point+2] - samp_nonoff_sensor[i_point+1]
                        slope_4 = samp_nonoff_sensor[i_point+3] - samp_nonoff_sensor[i_point+2]
                        slope_5 = samp_nonoff_sensor[i_point+4] - samp_nonoff_sensor[i_point+3]
                        slope_6 = samp_nonoff_sensor[i_point+5] - samp_nonoff_sensor[i_point+4]
                        # tough
                        if slope_1<0 and slope_3>0 and slope_4>0 and slope_5>0 and slope_6>0 and samp_nonoff_sensor[i_point]<0.4:    # do some change in the TP detection
                            # potential tough TP
                            TP_index_pot.append((i_point+start_adapt,-1))
                            continue
                        if slope_1>0 and slope_3<0 and slope_4<0 and slope_5<0 and slope_6<0 and samp_nonoff_sensor[i_point]>0.6:    # start point of adapt point
                            # potential peak TP
                            TP_index_pot.append((i_point+start_adapt,1))
                            continue
                        
                TP_index_pot = np.array(TP_index_pot,dtype = int)
                TP_index_det = []
                for i_TP in range(len(TP_index_pot)-1):
                    if TP_index_pot[i_TP+1,1]*TP_index_pot[i_TP,1] == -1:
                        TP_index_det.append(TP_index_pot[i_TP,:])
                    else:
                        continue

                TP_index_det = np.array(TP_index_det,dtype = int)
                
                # insert the startpoint in the TP index, label = 0
                TP_index_det_insert = np.insert(TP_index_det,0,[start_adapt,0],axis = 0)
                # print(TP_index_det_insert)
                
                TP_diff = np.max(np.diff(TP_index_det_insert[:,0]))
                # print(TP_diff)
                
                TP_diff_indice = np.argmax(np.diff(TP_index_det[:,0]))
                last_start = start_adapt
                start_adapt = int(TP_index_det[TP_diff_indice,0])

            TP_diff = np.diff(TP_index_det[:,0])[1:]   # without the diff of the startpoint
            
            # assum: peak -> Ex start, touth -> In start --- short Interval: tough to peak    
            # first detected TP is peak
            if  TP_index_det[1,1] == 1:
                TtoP_list = [TP_diff[x] for x in range(len(TP_diff)) if x %2 != 0]
                PtoT_list = [TP_diff[x] for x in range(len(TP_diff)) if x %2 == 0]
                if len(TtoP_list) != 0 and len(TtoP_list) != 0:
                    TtoP = sum(TtoP_list)/len(TtoP_list)
                    PtoT = sum(PtoT_list)/len(PtoT_list)
                    if TtoP > PtoT:
                        #print(str(i_file)+ ' sensor ' + str(i_match)+' should flip')
                        Inverse_sensor.append(i_match)
                else:
                    CC_match_ = np.delete(CC_match,np.where(CC_match == i_match))
                    continue
                    
            if  TP_index_det[1,1] == -1:
                TtoP_list = [TP_diff[x] for x in range(len(TP_diff)) if x %2 == 0]
                PtoT_list = [TP_diff[x] for x in range(len(TP_diff)) if x %2 != 0]
                if len(TtoP_list) != 0 and len(TtoP_list) != 0:
                    TtoP = sum(TtoP_list)/len(TtoP_list)
                    PtoT = sum(PtoT_list)/len(PtoT_list)
                    if TtoP > PtoT:
                        #print(str(i_file)+ ' sensor ' + str(i_match)+' should flip')
                        Inverse_sensor.append(i_match)                    
                else:
                    CC_match_ = np.delete(CC_match,np.where(CC_match == i_match))
                    continue
        
        sensor_train = []
        if len(CC_match_)>0:
            test_sample_num = np.random.randint(len(CC_match_))
            print(f'The selected testset is from {i_file}, sensor No.{CC_match_[test_sample_num]+1}.')
            test_sample = input_data[:, CC_match_[test_sample_num]]
            testset.append(test_sample)
            CC_match_ = np.delete(CC_match_, test_sample_num)
            
            for i_match_ in CC_match_:

                samp_Sav_filter_win =savgol_filter(sensors_value[:,i_match_],win_len,sav_order,mode='nearest')
                samp_baseline_sensor = baseline_als(samp_Sav_filter_win,lam,p)
                samp_nonoff_sensor = samp_Sav_filter_win - samp_baseline_sensor
        
                low_samp = np.quantile(samp_nonoff_sensor,0.25)
                up_samp = np.quantile(samp_nonoff_sensor,0.75)
                IQR = up_samp - low_samp
                
                samp_norm = (samp_nonoff_sensor + 2*IQR) / 4 / IQR
                samp_norm = np.clip(samp_norm,0,1)
            
                if i_match_ in Inverse_sensor:
                    samp_norm = -samp_norm + 1
                sensor_train.append(samp_norm)
                print('Gathered sensor File '+str(i_file)+' sensor '+str(i_match_))
            sensor_train = np.array(sensor_train).reshape(len(CC_match_),-1)
            df = pd.DataFrame(data = sensor_train)
                
            filepath_ = filepath+'/NormFile'+str(i_file_index)+'_train.csv'
            df.to_csv(filepath_,index = False,mode = 'w', header = False)
        else:
            continue

save_path_test = './splited_data/test_set.csv'
with open(save_path_test, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(testset)

The selected testset is from ./RawData_Baby_Sensorpatch2/2021-09-08_15-21-54_2021-09-08_15-24-06_sensor_input_5000_1631107314600325.csv, sensor No.23.
Gathered sensor File ./RawData_Baby_Sensorpatch2/2021-09-08_15-21-54_2021-09-08_15-24-06_sensor_input_5000_1631107314600325.csv sensor 19
Gathered sensor File ./RawData_Baby_Sensorpatch2/2021-09-08_15-21-54_2021-09-08_15-24-06_sensor_input_5000_1631107314600325.csv sensor 21
Gathered sensor File ./RawData_Baby_Sensorpatch2/2021-09-08_15-21-54_2021-09-08_15-24-06_sensor_input_5000_1631107314600325.csv sensor 25
Gathered sensor File ./RawData_Baby_Sensorpatch2/2021-09-08_15-21-54_2021-09-08_15-24-06_sensor_input_5000_1631107314600325.csv sensor 34
The selected testset is from ./RawData_Baby_Sensorpatch2/2021-09-08_15-21-54_2021-09-08_15-30-40_sensor_input_5000_1631107314600325.csv, sensor No.2.
Gathered sensor File ./RawData_Baby_Sensorpatch2/2021-09-08_15-21-54_2021-09-08_15-30-40_sensor_input_5000_1631107314600325.csv sensor 21
Gathered 

In [6]:
df = pd.read_csv(i_file).to_numpy()
df

array([[657490837,      4095,       471, ...,      1795,      4095,
             3860],
       [657517839,      4095,       475, ...,      1772,      4095,
             3838],
       [657543838,      4095,       484, ...,      1794,      4095,
             3840],
       ...,
       [725370493,      4095,      2483, ...,      2057,      1435,
             2483],
       [725397493,      4095,      2476, ...,      2117,      1403,
             2495],
       [725424495,      4095,      2447, ...,      2001,      1374,
             2496]], dtype=int64)

In [9]:
import numpy as np
import pandas as pd
import os
import time

import matplotlib.pyplot as plt

from scipy.signal import savgol_filter

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM,Bidirectional

from PreparingMethod import TP_detect,baseline_als

file_list = ["./RawData_Baby_Sensorpatch2/2021-09-08_15-21-54_sensor_input_23518_1631107314600325.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_15-32-19_sensor_input_22980_1631107939624088.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_15-42-30_sensor_input_22561_1631108550428185.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_15-52-28_sensor_input_21938_1631109148920218.csv",
             "./RawData_Baby_Sensorpatch2/2021-09-08_16-02-09_sensor_input_27586_1631109729643834.csv",]

# baseline
lam = 10e3
p = 0.5
# savFilter
win_len = 9
sav_order = 5
# adapt window length is 300
adapt_Window = 300
i_file_index = 0
#filepath = 'F:/PyCases/Jupter/Ventilator/LSTM_training/Train_data/traindata.csv'
#os.remove(filepath)
filepath = './Train_data'

for i_file in file_list:
    i_file_index += 1
    input_data = pd.read_csv(i_file).to_numpy()

    sensors_value = np.clip(input_data[:,1:],a_min = 0, a_max = 4095)    # 36sensors value
    timelineinsec = input_data[:,0]/1000000    # time in second, ndarray

    num_messdata, num_sensors = sensors_value.shape

    time_diff = np.diff(timelineinsec).mean()   # get the mean value of the deita time(in two sensor signal)
    sample_rate = 1/time_diff        # sample rate, Hz ~38Hz


    FC_peak_all, FC_tough_all, CC_all = [],[],[]
    for i_sensor in range(num_sensors):
        Sav_filter_win =savgol_filter(sensors_value[:,i_sensor],win_len,sav_order,mode='nearest')
        baseline_sensor = baseline_als(Sav_filter_win,lam,p)
        norm_sensor = Sav_filter_win - baseline_sensor
        FC_peak_index,FC_tough_index,FC_peak,FC_tough,CC = TP_detect(norm_sensor)
    
        FC_peak_all.append(FC_peak)
        FC_tough_all.append(FC_tough)
        CC_all.append(CC)

    CC_all = np.array(CC_all,dtype = int).reshape(-1)

    # according to the respiration frequency 1-2Hz,
    upbreath_freq = 1.5     # the max breath freq by infants
    lowbreath_freq = 0.8    # incase Artifact or Apenea, the min breath freq by infants

    # boundary for CC, Hz
    lowbound_CC = (2 * lowbreath_freq * 2) * num_messdata / sample_rate  # 2 means peak and tough
    upbound_CC =  (2 * upbreath_freq * 2) * num_messdata / sample_rate   # can max 2 times of the max breath frequency allowable
    up_CC = np.arange(num_sensors)[CC_all<upbound_CC]
    low_CC = np.arange(num_sensors)[CC_all>lowbound_CC]

    CC_match = np.array([x for x in low_CC if x in up_CC])     # selected sensor in CC boundary

    if len(CC_match) < 1:
        continue
    else:
        TP_index_all = []
        last_start_all = []
        Inverse_sensor = []
        CC_match_ = CC_match.copy()
        
        for i_match in CC_match:
            
            start_adapt = 0
            TP_diff = 50
            while TP_diff > 30:
                samp_Sav_filter_win =savgol_filter(sensors_value[:,i_match][start_adapt:start_adapt+adapt_Window],win_len,sav_order,mode='nearest')
                samp_baseline_sensor = baseline_als(samp_Sav_filter_win,lam,p)
                samp_nonoff_sensor = samp_Sav_filter_win - samp_baseline_sensor
                
                low_samp = np.quantile(samp_nonoff_sensor,0.25)
                up_samp = np.quantile(samp_nonoff_sensor,0.75)
                IQR = up_samp - low_samp
                
                samp_norm = (samp_nonoff_sensor + 2*IQR) / 4 / IQR
                samp_norm = np.clip(samp_norm,0,1)
                
                TP_index_pot = []
                for i_point in range(1,(adapt_Window-5)):
                    slope_1 = samp_nonoff_sensor[i_point] - samp_nonoff_sensor[i_point-1]
                    slope_2 = samp_nonoff_sensor[i_point+1] - samp_nonoff_sensor[i_point]
                # if change point
                    if slope_1*slope_2 > 0:
                        continue
                    else:
                        slope_3 = samp_nonoff_sensor[i_point+2] - samp_nonoff_sensor[i_point+1]
                        slope_4 = samp_nonoff_sensor[i_point+3] - samp_nonoff_sensor[i_point+2]
                        slope_5 = samp_nonoff_sensor[i_point+4] - samp_nonoff_sensor[i_point+3]
                        slope_6 = samp_nonoff_sensor[i_point+5] - samp_nonoff_sensor[i_point+4]
                        # tough
                        if slope_1<0 and slope_3>0 and slope_4>0 and slope_5>0 and slope_6>0 and samp_nonoff_sensor[i_point]<0.4:    # do some change in the TP detection
                            # potential tough TP
                            TP_index_pot.append((i_point+start_adapt,-1))
                            continue
                        if slope_1>0 and slope_3<0 and slope_4<0 and slope_5<0 and slope_6<0 and samp_nonoff_sensor[i_point]>0.6:    # start point of adapt point
                            # potential peak TP
                            TP_index_pot.append((i_point+start_adapt,1))
                            continue
                        
                TP_index_pot = np.array(TP_index_pot,dtype = int)
                TP_index_det = []
                for i_TP in range(len(TP_index_pot)-1):
                    if TP_index_pot[i_TP+1,1]*TP_index_pot[i_TP,1] == -1:
                        TP_index_det.append(TP_index_pot[i_TP,:])
                    else:
                        continue

                TP_index_det = np.array(TP_index_det,dtype = int)
                
                # insert the startpoint in the TP index, label = 0
                TP_index_det_insert = np.insert(TP_index_det,0,[start_adapt,0],axis = 0)
                print(TP_index_det_insert)
                
                TP_diff = np.max(np.diff(TP_index_det_insert[:,0]))
                print(TP_diff)
                
                TP_diff_indice = np.argmax(np.diff(TP_index_det[:,0]))
                last_start = start_adapt
                start_adapt = int(TP_index_det[TP_diff_indice,0])

            TP_diff = np.diff(TP_index_det[:,0])[1:]   # without the diff of the startpoint
            
            # assum: peak -> Ex start, touth -> In start --- short Interval: tough to peak    
            # first detected TP is peak
            if  TP_index_det[1,1] == 1:
                TtoP_list = [TP_diff[x] for x in range(len(TP_diff)) if x %2 != 0]
                PtoT_list = [TP_diff[x] for x in range(len(TP_diff)) if x %2 == 0]
                if len(TtoP_list) != 0 and len(TtoP_list) != 0:
                    TtoP = sum(TtoP_list)/len(TtoP_list)
                    PtoT = sum(PtoT_list)/len(PtoT_list)
                    if TtoP > PtoT:
                        #print(str(i_file)+ ' sensor ' + str(i_match)+' should flip')
                        Inverse_sensor.append(i_match)
                else:
                    CC_match_ = np.delete(CC_match,np.where(CC_match == i_match))
                    continue
                    
            if  TP_index_det[1,1] == -1:
                TtoP_list = [TP_diff[x] for x in range(len(TP_diff)) if x %2 == 0]
                PtoT_list = [TP_diff[x] for x in range(len(TP_diff)) if x %2 != 0]
                if len(TtoP_list) != 0 and len(TtoP_list) != 0:
                    TtoP = sum(TtoP_list)/len(TtoP_list)
                    PtoT = sum(PtoT_list)/len(PtoT_list)
                    if TtoP > PtoT:
                        #print(str(i_file)+ ' sensor ' + str(i_match)+' should flip')
                        Inverse_sensor.append(i_match)                    
                else:
                    CC_match_ = np.delete(CC_match,np.where(CC_match == i_match))
                    continue
        
        sensor_train = []
        if len(CC_match_)>0:
            for i_match_ in CC_match_:

                samp_Sav_filter_win =savgol_filter(sensors_value[:,i_match_],win_len,sav_order,mode='nearest')
                samp_baseline_sensor = baseline_als(samp_Sav_filter_win,lam,p)
                samp_nonoff_sensor = samp_Sav_filter_win - samp_baseline_sensor
        
                low_samp = np.quantile(samp_nonoff_sensor,0.25)
                up_samp = np.quantile(samp_nonoff_sensor,0.75)
                IQR = up_samp - low_samp
                
                samp_norm = (samp_nonoff_sensor + 2*IQR) / 4 / IQR
                samp_norm = np.clip(samp_norm,0,1)
            
                if i_match_ in Inverse_sensor:
                    samp_norm = -samp_norm + 1
                sensor_train.append(samp_norm)
                print('Gathered sensor File '+str(i_file)+' sensor '+str(i_match_))
            sensor_train = np.array(sensor_train).reshape(len(CC_match_),-1)
            df = pd.DataFrame(data = sensor_train)
                
            filepath_ = filepath+'/NormFile'+str(i_file_index)+'_train.csv'
            df.to_csv(filepath_,index = False,mode = 'w', header = False)
        else:
            continue

[[  0   0]
 [ 13   1]
 [ 71  -1]
 [137   1]
 [162  -1]
 [176   1]
 [193  -1]
 [204   1]
 [225  -1]
 [237   1]
 [252  -1]
 [263   1]
 [281  -1]]
66
[[ 71   0]
 [137   1]
 [162  -1]
 [176   1]
 [193  -1]
 [204   1]
 [225  -1]
 [237   1]
 [252  -1]
 [263   1]
 [279  -1]
 [291   1]
 [311  -1]
 [322   1]
 [341  -1]]
66
[[137   0]
 [162  -1]
 [176   1]
 [193  -1]
 [204   1]
 [225  -1]
 [237   1]
 [252  -1]
 [263   1]
 [279  -1]
 [291   1]
 [311  -1]
 [322   1]
 [341  -1]
 [354   1]
 [378  -1]
 [389   1]
 [410  -1]]
25
[[  0   0]
 [ 31   1]
 [104  -1]
 [122   1]
 [162  -1]
 [176   1]
 [193  -1]
 [203   1]
 [224  -1]
 [236   1]
 [252  -1]
 [263   1]
 [280  -1]]
73
[[ 31   0]
 [104  -1]
 [122   1]
 [162  -1]
 [176   1]
 [193  -1]
 [203   1]
 [224  -1]
 [236   1]
 [252  -1]
 [263   1]
 [280  -1]
 [291   1]
 [312  -1]]
73
[[122   0]
 [125   1]
 [162  -1]
 [176   1]
 [193  -1]
 [203   1]
 [224  -1]
 [236   1]
 [252  -1]
 [263   1]
 [280  -1]
 [291   1]
 [312  -1]
 [322   1]
 [343  -1]
 [354   1]
 

[[  0   0]
 [ 10  -1]
 [ 19   1]
 [ 36  -1]
 [ 45   1]
 [ 70  -1]
 [ 83   1]
 [113  -1]
 [126   1]
 [149  -1]
 [159   1]
 [182  -1]
 [193   1]
 [214  -1]
 [226   1]
 [247  -1]
 [259   1]
 [277  -1]]
30
[[  0   0]
 [ 13  -1]
 [ 19   1]
 [ 35  -1]
 [ 45   1]
 [ 69  -1]
 [ 82   1]
 [111  -1]
 [126   1]
 [149  -1]
 [159   1]
 [181  -1]
 [193   1]
 [214  -1]
 [226   1]
 [246  -1]
 [258   1]
 [277  -1]]
29
[[  0   0]
 [ 13  -1]
 [ 19   1]
 [ 36  -1]
 [ 45   1]
 [ 68  -1]
 [ 81   1]
 [110  -1]
 [125   1]
 [149  -1]
 [159   1]
 [181  -1]
 [192   1]
 [213  -1]
 [226   1]
 [247  -1]
 [257   1]
 [277  -1]]
29
[[  0   0]
 [ 19   1]
 [ 35  -1]
 [ 45   1]
 [ 68  -1]
 [ 93   1]
 [112  -1]
 [125   1]
 [148  -1]
 [158   1]
 [180  -1]
 [192   1]
 [213  -1]
 [226   1]
 [245  -1]
 [257   1]
 [276  -1]]
25
[[  0   0]
 [  1  -1]
 [ 19   1]
 [ 35  -1]
 [ 45   1]
 [ 68  -1]
 [ 81   1]
 [110  -1]
 [125   1]
 [148  -1]
 [158   1]
 [181  -1]
 [192   1]
 [213  -1]
 [225   1]
 [243  -1]
 [258   1]
 [276  -1]]
29
[

[[209   0]
 [243  -1]
 [251   1]
 [272  -1]
 [292   1]
 [306  -1]
 [323   1]
 [338  -1]
 [352   1]
 [370  -1]
 [381   1]
 [401  -1]
 [411   1]
 [430  -1]
 [441   1]
 [462  -1]
 [472   1]]
34
[[251   0]
 [272  -1]
 [292   1]
 [306  -1]
 [323   1]
 [338  -1]
 [352   1]
 [370  -1]
 [381   1]
 [401  -1]
 [411   1]
 [431  -1]
 [441   1]
 [462  -1]
 [472   1]
 [494  -1]
 [504   1]
 [530  -1]]
26
[[  0   0]
 [ 18  -1]
 [ 31   1]
 [ 48  -1]
 [ 60   1]
 [ 89  -1]
 [109   1]
 [134  -1]
 [146   1]
 [167  -1]
 [178   1]
 [199  -1]
 [209   1]
 [243  -1]
 [251   1]
 [272  -1]]
34
[[209   0]
 [210   1]
 [243  -1]
 [251   1]
 [272  -1]
 [290   1]
 [312  -1]
 [323   1]
 [341  -1]
 [352   1]
 [371  -1]
 [381   1]
 [401  -1]
 [411   1]
 [431  -1]
 [441   1]
 [462  -1]
 [472   1]]
33
[[210   0]
 [243  -1]
 [251   1]
 [272  -1]
 [290   1]
 [312  -1]
 [323   1]
 [341  -1]
 [352   1]
 [371  -1]
 [381   1]
 [401  -1]
 [411   1]
 [431  -1]
 [441   1]
 [462  -1]
 [472   1]
 [495  -1]]
33
[[472   0]
 [473   1]
 